In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import validation_curve
from sklearn.metrics import make_scorer
from sklearn import metrics as mt
from sklearn.metrics import f1_score as f1
from sklearn.svm import SVC
import seaborn as sns
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import os
from random import sample

## Import dataset (NO CV)

In [21]:
#training files
df_etd_training = pd.read_csv("../Data/chat_pattern/training/etd_pattern_training.csv")
df_ps_training = pd.read_csv("../Data/chat_pattern/training/ps_pattern_training.csv")
df_ngram_training = pd.read_csv("../Data/chat_pattern/training/ngram_training.csv")
df_pos_training = pd.read_csv("../Data/chat_pattern/training/pos_training.csv")

#testing files
df_etd_testing = pd.read_csv("../Data/chat_pattern/testing/etd_pattern_testing.csv")
df_ps_testing = pd.read_csv("../Data/chat_pattern/testing/ps_pattern_testing.csv")
df_ngram_testing = pd.read_csv("../Data/chat_pattern/testing/ngram_testing.csv")
df_pos_testing = pd.read_csv("../Data/chat_pattern/testing/pos_testing.csv")

#label
df_label_training = pd.read_csv("../Data/chat_pattern/chat_annotation_1000_pos.csv")
df_label_testing = pd.read_csv("../Data/chat_pattern/chat_testing_200.csv")

In [ ]:
X_etd_train = df_etd_training.values
X_ps_train = df_ps_training.values
X_ngram_train = df_ngram_training.values
X_pos_train = df_pos_training.values

X_etd_ngram_train = np.hstack((X_etd_train, X_ngram_train))
X_ps_ngram_train = np.hstack((X_ps_train, X_ngram_train))
X_etd_pos_train = np.hstack((X_etd_train, X_pos_train))
X_ps_pos_train = np.hstack((X_ps_train, X_pos_train))
X_ngram_pos_train = np.hstack((X_ngram_train, X_pos_train))

X_all_etd_train = np.hstack((X_etd_ngram_train, X_pos_train))
X_all_ps_train = np.hstack((X_ps_ngram_train, X_pos_train))

In [83]:
X_etd_test = df_etd_testing.values
X_ps_test = df_ps_testing.values
X_ngram_test = df_ngram_testing.values
X_pos_test = df_pos_testing.values


X_etd_ngram_test = np.hstack((X_etd_test, X_ngram_test))
X_etd_pos_test = np.hstack((X_etd_test, X_pos_test))
X_ngram_pos_test = np.hstack((X_ngram_test, X_pos_test))
X_ps_ngram_test = np.hstack((X_ps_test, X_ngram_test))
X_ps_pos_test = np.hstack((X_ps_test, X_pos_test))

X_all_etd_test = np.hstack((X_etd_ngram_test, X_pos_test))
X_all_ps_test = np.hstack((X_ngram_pos_test, X_pos_test))

In [65]:
y_etd_train = df_label_training["y_ETD"].values
y_ps_train = df_label_training["y_PS"].values
y_etd_test = df_label_testing["y_ETD"].values
y_ps_test = df_label_testing["y_PS"].values

## Prepare dataset for cross-validation (CV)

### Train Test Split (10-fold)

In [4]:
df_all = pd.read_csv("../Data/chat_pattern/chat_test_missing_1000.csv")
df_all.head()

,source,issue,PS_sent,predicted_PS,ETD_sent,predicted_ETD,y_PS,y'_PS,Acc_PS,y_ETD,...,Acc_ETD,openAI_PS,Inferred_PS_pattern,y''_PS,ACC_AI_PS,openAI_ETD,Inferred_ETD_pattern,coment,y''_ETD,ACC_AI_ETD
0,angular,"Hi, when I am trying to apply class dynamicall...",T2: it is messing up my material css i.e I am ...,"PS_NEG_VERB,PS_NEG_AUX_ADV_ADJ,PS_NEG_AUX_VERB,",T1: when I am trying to apply class dynamicall...,"ETD_TRYING_TO,",0,0,1,0,...,1,"The sentence """""" it is messing up my material ...",PS_NEG_VERB,1.0,0.0,"The sentence """""" when I am trying to apply cla...",NaN,NaN,1.0,0.0
1,angular,is it possible to get a RouteConfig matched ag...,NaN,NaN,T1: I'm trying to create a Breadcrumb componen...,"ETD_BE_POSSIBLE_TO,ETD_TRYING_TO,ETD_WOULD_LIKE,",1,1,1,0,...,1,No patterns found. The answer is NO.,NaN,0.0,0.0,"The sentence """""" I'm trying to create a Breadc...",NaN,NaN,1.0,0.0
2,angular,I have angular running from a .Net Core server...,T1: The server won't even load the app (by des...,"PS_NEG_AUX_VERB,",What's the best way for the angular app to get...,NaN,0,0,1,0,...,0,No patterns found. The answer is NO.,NaN,0.0,1.0,"The sentence """""" What's the best way for the a...",new,NaN,1.0,0.0
3,angular,"Hello everyone, I want to do sub menu with sea...",T2: but i have problem in sub menu,"PS_PROBLEM,",T1: I want to do sub menu with searh like in g...,"ETD_WANT_TO,ETD_CAN_QUESTION,",0,0,1,0,...,1,"The sentence """""" i have problem in sub menu wi...","""problem"" is the [error-term]",1.0,0.0,"The sentence ""i have problem in sub menu with ...",new,NaN,1.0,0.0
4,angular,I'm struggling with getting this logic\n ...,T1: I'm struggling with getting this logic,"PS_STRUGGLING,PS_VERB_NO,",T2: I wanted to set a default value to the dro...,"ETD_WANT_TO,",0,0,1,0,...,1,"The sentence ""I'm struggling with getting this...",NaN,1.0,0.0,"The sentence """"""I wanted to set a default valu...",NaN,NaN,1.0,0.0


In [5]:
df_random = df_all.sample(n=len(df_all),ignore_index=True)

In [6]:
cv = 10

train_path = "../Data/chat_pattern/gpt_iterative/cross_validate/"
for i in range(cv):
    df_split = np.array_split(df_random, cv)
    df_test = df_split[i]
    df_val = df_split[len(df_split)-i-1]
    frame_train = [df_split[index] for index in range(len(df_split)) if index != i and index != (len(df_split)-i-1)]
    df_train = pd.concat(frame_train)
    
    if not os.path.exists(train_path+str(i)+"/"):
        os.makedirs(train_path+str(i)+"/")
        
    df_train.to_csv(train_path+str(i)+"/"+"train_800.csv",index = None)
    df_val.to_csv(train_path+str(i)+"/"+"val_100.csv",index = None)
    df_test.to_csv(train_path+str(i)+"/"+"test_100.csv",index = None)

### Pattern Feature Extraction

In [7]:
def pattern_matrix(df, col):
    pattern_code = set()
    for pattern_str in df[col]:
        if pattern_str == 0:
            continue
        patterns = pattern_str.split(",")
        for pattern in patterns:
            if len(pattern) != 0:
                pattern_code.add(pattern.strip())
                
    pattern_list = []
    for i in range(len(df)):
        pattern_str = df.loc[i][col]
        pattern_val = dict.fromkeys(pattern_code,0)
        if pattern_str != 0:
            patterns = pattern_str.split(",")
            for pattern in patterns:
                if len(pattern) != 0:
                    pattern_val[pattern.strip()] += 1
    
        pattern_list.append(pattern_val)
    
    
    return pattern_code,pd.DataFrame.from_dict(pattern_list)

In [8]:
def pattern_matrix_testing(pattern_code, df, col):            
    pattern_list = []
    for i in range(len(df)):
        pattern_str = df.loc[i][col]
        pattern_val = dict.fromkeys(pattern_code,0)
        if pattern_str != 0:
            patterns = pattern_str.split(",")
            for pattern in patterns:
                if len(pattern) != 0 and pattern != '0' and pattern in pattern_code:
                    pattern_val[pattern.strip()] += 1
    
        pattern_list.append(pattern_val)
    
    
    return pd.DataFrame.from_dict(pattern_list)

In [12]:
df = pd.read_csv("../Data/chat_pattern/chat_annotation_1000_pos.csv")
df["predicted_ETD"].fillna(0,inplace=True)
df["predicted_PS"].fillna(0,inplace=True)
df

,issue,POS,ETD_sent,Pattern for ETD,predicted_ETD,Comment_ETD,y_ETD,y'_ETD,accuracy_ETD,PS_sent,Pattern for PS,predicted_PS,Comment_PS,y_PS,y'_PS,accuracy_PS,TS_sent
0,"Hey guys, I'm trying to set a property in my c...","Hey-UH guys-NNS , I-PRP 'm-VBP trying-VBG to-T...",T1: I'm trying to set a property in my compone...,ETD_TRYING_TO,"ETD_TRYING_TO,",NaN,1,1,1,But everywhere I try to do it messes with chan...,PS_PROBLEM,"PS_PROBLEM,",NaN,1,1,1,NaN
1,"Hi, I'm following tutorials from angular.io an...","Hi-UH , I-PRP 'm-VBP following-VBG tutorials-N...",Can I configure building anyhow that I don't h...,ETD_CAN_QUESTION,"ETD_CAN_QUESTION,",NaN,1,1,1,T1: When I move it somewhere else I have probl...,PS_PROBLEM,"PS_NEG_ADV_ADJ,PS_NEG_AUX_VERB,PS_VERB_ERROR,P...",NaN,1,1,1,NaN
2,Trying to come up with a generic pattern valid...,Trying-VBG to-TO come-VB up-RP with-IN a-DT ge...,T1: Trying to come up with a generic pattern v...,ETD_TRYING_TO,"ETD_INSTEAD_OF_EXP_BEHAVIOR,ETD_TRYING_TO,",NaN,1,1,1,T2: but running into an issue of only getting ...,PS_PROBLEM,"PS_PROBLEM,PS_NO_NOUN,PS_ONLY,",NaN,1,1,1,NaN
3,I'm sure this has been asked plenty before but...,I-PRP 'm-VBP sure-JJ this-DT has-VBZ been-VBN ...,T1: now looking to move to Angular2,ETD_LOOKING_TO,"ETD_LOOKING_TO,",NaN,1,1,1,NaN,NaN,0,NaN,0,0,1,NaN
4,Does anyone know why there is anngSubmitEventE...,Does-VBZ anyone-NN know-VB why-WRB there-EX is...,NaN,NaN,"ETD_LOOKING_TO,",NaN,0,1,0,T1: But I don't see any reason why there is a ...,"PS_NEG_AUX_VERB,PS_ONLY","PS_NEG_AUX_VERB,",NaN,1,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Hello, a transaction has given me gas error.Bu...","Hello-UH , a-DT transaction-NN has-VBZ given-V...",NaN,NaN,"ETD_CAN_QUESTION,",NaN,0,1,0,a transaction has given me gas error,"PS_NEG_AUX_VERB,PS_NEG_ADV_ADJ,","PS_NEG_AUX_VERB,PS_NEG_ADV_ADJ,",NaN,1,1,1,NaN
996,Hello [<-CODE->] Here I have [<-CODE->] Erro...,Hello-UH [--LRB- <-SYM --HYPH CODE-NN --HYPH >...,NaN,NaN,0,NaN,0,0,1,"Here I have [<-CODE->] Error:(10, 16)","PS_NEG_AUX_ADV_ADJ,PS_PROBLEM,PS_NO_NOUN,PS_VE...","PS_NEG_AUX_ADV_ADJ,PS_PROBLEM,PS_NO_NOUN,PS_VE...",NaN,1,1,1,NaN
997,org.openqa.selenium.WebDriverException: org.ap...,org.openqa.selenium.WebDriverException-NN : or...,NaN,NaN,0,NaN,0,0,1,failed: Connection refused: connect Getting th...,"PS_PROBLEM,PS_NEG_AUX_VERB,PS_NEG_VERB,PS_ERRO...","PS_PROBLEM,PS_NEG_AUX_VERB,PS_NEG_VERB,PS_ERRO...",NaN,1,1,1,NaN
998,Not sure what this error is about: Error resp...,Not-RB sure-JJ what-WP this-DT error-NN is-VBZ...,NaN,NaN,"ETD_CAN_QUESTION,ETD_FOR_EXAMPLE,",NaN,0,1,0,Error response from daemon: rpc error,"PS_VERB_ERROR,PS_ERROR_NOUN_PHRASE,","PS_VERB_ERROR,PS_ERROR_NOUN_PHRASE,",NaN,1,1,1,NaN


Prepare random issues if not use all issues from pattern discovery set

In [ ]:
## 30 random issues
df.sample(n=30).to_csv("../../openai/experiment/random_issue/PS/random_issues.csv",index = None)

In [ ]:
df_r = pd.read_csv("../../openai/experiment/random_issue/PS/ps_prompt_random.csv")
df_r.head()

In [ ]:
df_r.sample(frac = 1,ignore_index=True).to_csv("../../openai/experiment/random_issue/PS/random_25_issues.csv",index = None)

Feature extraction 

In [10]:
df = pd.read_csv("../Data/chat_pattern/gpt_iterative/random_issues.csv") # selected random issues for pattern discovery

In [11]:
etd_pattern_code,df_etd = pattern_matrix(df,"predicted_ETD")
ps_pattern_code,df_ps = pattern_matrix(df,"predicted_PS")

In [12]:
etd_pattern_code = {code for code in etd_pattern_code if code != '0'}
etd_pattern_code

{'ETD_CAN_QUESTION',
 'ETD_IN_ORDER_FOR',
 'ETD_SHOULD',
 'ETD_TRYING_TO',
 'ETD_WOULD_LIKE'}

In [13]:
ps_pattern_code = {code for code in ps_pattern_code if code != '0'}
ps_pattern_code

{'PS_ETD_TRY',
 'PS_NEG_ADV_ADJ',
 'PS_NEG_AUX_VERB',
 'PS_PROBLEM',
 'PS_PROBLEM_IS',
 'PS_STRUGGLING',
 'PS_VERB_ERROR'}

In [41]:
# for one file
df_infer = pd.read_csv("../Data/chat_pattern/gpt_iterative/inference/pattern_discovery/chat_annotation_995.csv")
df_infer.fillna(0,inplace=True)
df_etd_infer = pattern_matrix_testing(etd_pattern_code, df_infer,"predicted_ETD")
df_ps_infer = pattern_matrix_testing(ps_pattern_code, df_infer,"predicted_PS")

df_etd_infer.to_csv("../Data/chat_pattern/gpt_iterative/inference/pattern_discovery/etd_pattern_infer.csv",index =None)
df_ps_infer.to_csv("../Data/chat_pattern/gpt_iterative/inference/pattern_discovery/ps_pattern_infer.csv",index =None)

In [20]:
#for all cross-validate data
path = "../Data/chat_pattern/gpt_iterative/cross_validate/"

dir_list = os.listdir(path)

if ".DS_Store" in dir_list:
    dir_list.remove(".DS_Store")

for folder in dir_list:
    df_train = pd.read_csv(path+folder+"/train_800.csv")
    df_test = pd.read_csv(path+folder+"/test_100.csv")
    df_val = pd.read_csv(path+folder+"/val_100.csv")
    
    df_train.fillna(0,inplace=True)
    df_test.fillna(0,inplace=True)
    df_val.fillna(0,inplace=True)
    
    df_etd_train = pattern_matrix_testing(etd_pattern_code, df_train,"predicted_ETD")
    df_ps_train = pattern_matrix_testing(ps_pattern_code, df_train,"predicted_PS")
    df_etd_test = pattern_matrix_testing(etd_pattern_code, df_test,"predicted_ETD")
    df_ps_test = pattern_matrix_testing(ps_pattern_code, df_test,"predicted_PS")
    df_etd_val = pattern_matrix_testing(etd_pattern_code, df_val,"predicted_ETD")
    df_ps_val = pattern_matrix_testing(ps_pattern_code, df_val,"predicted_PS")
    
    if not os.path.exists(path+folder+"/train/"):
        os.makedirs(path+folder+"/train/")
    if not os.path.exists(path+folder+"/test/"):
        os.makedirs(path+folder+"/test/")
    if not os.path.exists(path+folder+"/val/"):
        os.makedirs(path+folder+"/val/")
        
    df_etd_train.to_csv(path+folder+"/train/etd_pattern_train.csv",index =None)
    df_ps_train.to_csv(path+folder+"/train/ps_pattern_train.csv",index =None)
    df_etd_test.to_csv(path+folder+"/test/etd_pattern_test.csv",index =None)
    df_ps_test.to_csv(path+folder+"/test/ps_pattern_test.csv",index =None)
    df_etd_val.to_csv(path+folder+"/val/etd_pattern_val.csv",index =None)
    df_ps_val.to_csv(path+folder+"/val/ps_pattern_val.csv",index =None)

### Feature Preparation

In [21]:
X_etd_train_list = []
X_ps_train_list = []
X_ngram_train_list = []
X_pos_train_list = []
X_etd_ngram_train_list = []
X_ps_ngram_train_list = []
X_etd_pos_train_list = []
X_ps_pos_train_list = []
X_ngram_pos_train_list = []
X_all_etd_train_list = []
X_all_ps_train_list = []

X_etd_test_list = []
X_ps_test_list = []
X_ngram_test_list = []
X_pos_test_list = []
X_etd_ngram_test_list = []
X_ps_ngram_test_list = []
X_etd_pos_test_list = []
X_ps_pos_test_list = []
X_ngram_pos_test_list = []
X_all_etd_test_list = []
X_all_ps_test_list = []

X_etd_val_list = []
X_ps_val_list = []
X_ngram_val_list = []
X_pos_val_list = []
X_etd_ngram_val_list = []
X_ps_ngram_val_list = []
X_etd_pos_val_list = []
X_ps_pos_val_list = []
X_ngram_pos_val_list = []
X_all_etd_val_list = []
X_all_ps_val_list = []

y_etd_train_list = []
y_ps_train_list = []
y_etd_test_list = []
y_ps_test_list = []
y_etd_val_list = []
y_ps_val_list = []

In [4]:
def prepare_10fold_data(path):
    dir_list = os.listdir(path)
    for folder in dir_list:
        #training files
        df_etd_train = pd.read_csv(path+folder+"/train/etd_pattern_train.csv")
        df_ps_train = pd.read_csv(path+folder+"/train/ps_pattern_train.csv")
        df_ngram_train = pd.read_csv(path+folder+"/train/ngram_train.csv")
        df_pos_train = pd.read_csv(path+folder+"/train/pos_train.csv")

        #test files
        df_etd_test = pd.read_csv(path+folder+"/test/etd_pattern_test.csv")
        df_ps_test = pd.read_csv(path+folder+"/test/ps_pattern_test.csv")
        df_ngram_test = pd.read_csv(path+folder+"/test/ngram_test.csv")
        df_pos_test = pd.read_csv(path+folder+"/test/pos_test.csv")
        
        #validation files
        df_etd_val = pd.read_csv(path+folder+"/val/etd_pattern_val.csv")
        df_ps_val = pd.read_csv(path+folder+"/val/ps_pattern_val.csv")
        df_ngram_val = pd.read_csv(path+folder+"/val/ngram_val.csv")
        df_pos_val = pd.read_csv(path+folder+"/val/pos_val.csv")

        #label
        df_label_train = pd.read_csv(path+folder+"/train_800.csv")
        df_label_test = pd.read_csv(path+folder+"/test_100.csv")
        df_label_val = pd.read_csv(path+folder+"/val_100.csv")
        
        #train
        X_etd_train = df_etd_train.values
        X_ps_train = df_ps_train.values
        X_ngram_train = df_ngram_train.values
        X_pos_train = df_pos_train.values

        X_etd_ngram_train = np.hstack((X_etd_train, X_ngram_train))
        X_ps_ngram_train = np.hstack((X_ps_train, X_ngram_train))
        X_etd_pos_train = np.hstack((X_etd_train, X_pos_train))
        X_ps_pos_train = np.hstack((X_ps_train, X_pos_train))
        X_ngram_pos_train = np.hstack((X_ngram_train, X_pos_train))

        X_all_etd_train = np.hstack((X_etd_ngram_train, X_pos_train))
        X_all_ps_train = np.hstack((X_ps_ngram_train, X_pos_train))
        
        #test
        X_etd_test = df_etd_test.values
        X_ps_test = df_ps_test.values
        X_ngram_test = df_ngram_test.values
        X_pos_test = df_pos_test.values

        X_etd_ngram_test = np.hstack((X_etd_test, X_ngram_test))
        X_etd_pos_test = np.hstack((X_etd_test, X_pos_test))
        X_ngram_pos_test = np.hstack((X_ngram_test, X_pos_test))
        X_ps_ngram_test = np.hstack((X_ps_test, X_ngram_test))
        X_ps_pos_test = np.hstack((X_ps_test, X_pos_test))

        X_all_etd_test = np.hstack((X_etd_ngram_test, X_pos_test))
        X_all_ps_test = np.hstack((X_ps_ngram_test, X_pos_test))
        
        #val
        X_etd_val = df_etd_val.values
        X_ps_val = df_ps_val.values
        X_ngram_val = df_ngram_val.values
        X_pos_val = df_pos_val.values

        X_etd_ngram_val = np.hstack((X_etd_val, X_ngram_val))
        X_etd_pos_val = np.hstack((X_etd_val, X_pos_val))
        X_ngram_pos_val = np.hstack((X_ngram_val, X_pos_val))
        X_ps_ngram_val = np.hstack((X_ps_val, X_ngram_val))
        X_ps_pos_val = np.hstack((X_ps_val, X_pos_val))

        X_all_etd_val = np.hstack((X_etd_ngram_val, X_pos_val))
        X_all_ps_val = np.hstack((X_ps_ngram_val, X_pos_val))
        
        y_etd_train = df_label_train["y_ETD"].values
        y_ps_train = df_label_train["y_PS"].values
        y_etd_test = df_label_test["y_ETD"].values
        y_ps_test = df_label_test["y_PS"].values
        y_etd_val = df_label_val["y_ETD"].values
        y_ps_val = df_label_val["y_PS"].values
        
        X_etd_train_list.append(X_etd_train)
        X_ps_train_list.append(X_ps_train)
        X_ngram_train_list.append(X_ngram_train)
        X_pos_train_list.append(X_pos_train)
        X_etd_ngram_train_list.append(X_etd_ngram_train)
        X_ps_ngram_train_list.append(X_ps_ngram_train)
        X_etd_pos_train_list.append(X_etd_pos_train)
        X_ps_pos_train_list.append(X_ps_pos_train)
        X_ngram_pos_train_list.append(X_ngram_pos_train)
        X_all_etd_train_list.append(X_all_etd_train)
        X_all_ps_train_list.append(X_all_ps_train)
        
        X_etd_test_list.append(X_etd_test)
        X_ps_test_list.append(X_ps_test)
        X_ngram_test_list.append(X_ngram_test)
        X_pos_test_list.append(X_pos_test)
        X_etd_ngram_test_list.append(X_etd_ngram_test)
        X_ps_ngram_test_list.append(X_ps_ngram_test)
        X_etd_pos_test_list.append(X_etd_pos_test)
        X_ps_pos_test_list.append(X_ps_pos_test)
        X_ngram_pos_test_list.append(X_ngram_pos_test)
        X_all_etd_test_list.append(X_all_etd_test)
        X_all_ps_test_list.append(X_all_ps_test)
        
        X_etd_val_list.append(X_etd_val)
        X_ps_val_list.append(X_ps_val)
        X_ngram_val_list.append(X_ngram_val)
        X_pos_val_list.append(X_pos_val)
        X_etd_ngram_val_list.append(X_etd_ngram_val)
        X_ps_ngram_val_list.append(X_ps_ngram_val)
        X_etd_pos_val_list.append(X_etd_pos_val)
        X_ps_pos_val_list.append(X_ps_pos_val)
        X_ngram_pos_val_list.append(X_ngram_pos_val)
        X_all_etd_val_list.append(X_all_etd_val)
        X_all_ps_val_list.append(X_all_ps_val)
        
        y_etd_train_list.append(y_etd_train)
        y_ps_train_list.append(y_ps_train)
        y_etd_test_list.append(y_etd_test)
        y_ps_test_list.append(y_ps_test)
        y_etd_val_list.append(y_etd_val)
        y_ps_val_list.append(y_ps_val)


In [5]:
prepare_10fold_data("../Data/chat_pattern/cross_validate/")

In [24]:
def prepare_iterative_data(path):
    dir_list = os.listdir(path)
    if ".DS_Store" in dir_list:
        dir_list.remove(".DS_Store")
    
    for folder in dir_list:
        #training files
        df_etd_train = pd.read_csv(path+folder+"/train/etd_pattern_train.csv")
        df_ps_train = pd.read_csv(path+folder+"/train/ps_pattern_train.csv")

        #test files
        df_etd_test = pd.read_csv(path+folder+"/test/etd_pattern_test.csv")
        df_ps_test = pd.read_csv(path+folder+"/test/ps_pattern_test.csv")
        
        #validation files
        df_etd_val = pd.read_csv(path+folder+"/val/etd_pattern_val.csv")
        df_ps_val = pd.read_csv(path+folder+"/val/ps_pattern_val.csv")

        #label
        df_label_train = pd.read_csv(path+folder+"/train_800.csv")
        df_label_test = pd.read_csv(path+folder+"/test_100.csv")
        df_label_val = pd.read_csv(path+folder+"/val_100.csv")
        
        #train
        X_etd_train = df_etd_train.values
        X_ps_train = df_ps_train.values
        
        #test
        X_etd_test = df_etd_test.values
        X_ps_test = df_ps_test.values
        
        #val
        X_etd_val = df_etd_val.values
        X_ps_val = df_ps_val.values
        
        y_etd_train = df_label_train["y_ETD"].values
        y_ps_train = df_label_train["y_PS"].values
        y_etd_test = df_label_test["y_ETD"].values
        y_ps_test = df_label_test["y_PS"].values
        y_etd_val = df_label_val["y_ETD"].values
        y_ps_val = df_label_val["y_PS"].values
        
        X_etd_train_list.append(X_etd_train)
        X_ps_train_list.append(X_ps_train)
        
        X_etd_test_list.append(X_etd_test)
        X_ps_test_list.append(X_ps_test)
        
        X_etd_val_list.append(X_etd_val)
        X_ps_val_list.append(X_ps_val)
        
        y_etd_train_list.append(y_etd_train)
        y_ps_train_list.append(y_ps_train)
        y_etd_test_list.append(y_etd_test)
        y_ps_test_list.append(y_ps_test)
        y_etd_val_list.append(y_etd_val)
        y_ps_val_list.append(y_ps_val)


In [25]:
prepare_iterative_data("../Data/chat_pattern/gpt_iterative/cross_validate/")

## Model Training

In [131]:
def train_evaluate(X_train, y_train, X_test, y_test):
    y_test_model = []
    yhat_model = []
    param_grid = {'C': np.linspace(0.001, 100, 20)}
    
    svc = SVC()
    grid_search = GridSearchCV(svc, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    print('CV Train score: {:.2f}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    
    predictions = grid_search.predict(X_test)
    precison = mt.precision_score(y_test, predictions)
    recall = mt.recall_score(y_test, predictions)
    score = mt.f1_score(y_test, predictions)
    
    print("precision:",round(precison,3),"recall:",round(recall,3),"F1:",round(score,3))
    
    for val in zip(y_test, predictions):
        yhat_model.append(val[1])
        y_test_model.append(val[0])
    
    return precison,recall,score, y_test_model, yhat_model

In [ ]:
def train_evaluate_2(X_train, y_train, X_test, y_test):
    y_test_model = []
    yhat_model = []
    param_grid = {'C': np.linspace(0.001, 100, 20)}
    
    svc = SVC()
    scoring = {'Precision': 'precision', 'Recall':'recall', 'F1': 'f1'}
    grid_search = GridSearchCV(svc, param_grid, cv=10, scoring = scoring, refit = "F1")
    
    #X_train = np.vstack((X_train, X_test))
    #y_train = np.concatenate((y_train, y_test))
    
    grid_search.fit(X_train, y_train)
    
    #print('CV Train score: {:.2f}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    
    results = grid_search.cv_results_
    
    precison = np.average(results["mean_test_Precision"])
    recall = np.average(results["mean_test_Recall"])
    score = np.average(results["mean_test_F1"])
    
    print("precision:",round(precison,3),"recall:",round(recall,3),"F1:",round(score,3))
    
    return results

In [32]:
def cross_validate(X_train_list, y_train_list, X_test_list, y_test_list, X_val_list, y_val_list):
    
    param_grid = {'C': np.linspace(0.001, 100, 20)}
#     precision_list = []
#     recall_list = []
#     f1_list = []
    yhat = []
    ytest = []
    clf = None
    
    for X_train,y_train,X_test,y_test,X_val,y_val in zip(X_train_list, y_train_list, X_test_list, y_test_list, X_val_list, y_val_list):
        svc = SVC()
        grid_search = GridSearchCV(svc, param_grid, cv=5)
        grid_search.fit(X_val, y_val)
    
        #print('CV Train score: {:.2f}'.format(grid_search.best_score_))
        print('Best parameters: {}'.format(grid_search.best_params_))
        
        clf = SVC(**grid_search.best_params_)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        
        for val in zip(y_test, predictions):
            yhat.append(val[1])
            ytest.append(val[0])
    
    precison = mt.precision_score(ytest, yhat)
    recall = mt.recall_score(ytest, yhat)
    score = mt.f1_score(ytest, yhat)
        
#         precision_list.append(precison)
#         recall_list.append(recall)
#         f1_list.append(score)
    
#     print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))
    print("precision:",round(precison,3),"recall:",round(recall,3),"F1:",round(score,3))
    
    return clf, ytest, yhat

## Evaluation and Inference

In [30]:
def result_save(input_path, yhat_ETD = None, yhat_PS = None):
    #merge all test folds
    test_list = [pd.read_csv(input_path+folder+"/test_100.csv") for folder in os.listdir(input_path) if folder != ".DS_Store"]
    df_test_all = pd.concat(test_list)
    
    df_final = pd.DataFrame()
    df_final["issue"] = df_test_all["issue"]
    df_final["y_PS"] = df_test_all["y_PS"]
    df_final["y_ETD"] = df_test_all["y_ETD"]
    if yhat_ETD != None:
        df_final["yhat_ETD"] = yhat_ETD
    if yhat_PS != None:
        df_final["yhat_PS"] = yhat_PS
    
    return df_final

In [47]:
def evaluation_report(clf, infer_path, pattern_path):
    df_data = pd.read_csv(pattern_path)
    X_infer = df_data.values
    y_prob = clf.predict(X_infer)
    y_prob_val = [np.max(y_prob[i]) for i in range(len(y_prob))]
    predicted_category_index = [np.argmax(y_prob[i]) for i in range(len(y_prob))]
#     y_pred = [class_list[index] for index in predicted_category_index]
    
    ## generate report
#     label_pred = np.zeros(len(y_pred))
    df_final = pd.read_csv(infer_path)
    df_final["y_prob"] = y_prob_val
    df_final["y_pred"] = predicted_category_index
    
    return df_final

In [33]:
# 1. ETD: pattern
clf1,etd_test, etd_hat = cross_validate(X_etd_train_list, y_etd_train_list, X_etd_test_list, y_etd_test_list, X_etd_val_list, y_etd_val_list)

Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
precision: 0.0 recall: 0.0 F1: 0.0


/Users/wangsimin/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
train_path = "../Data/chat_pattern/gpt_iterative/cross_validate/"
df_final = result_save(train_path, etd1)
df_final.to_csv("../Data/chat_pattern/gpt_iterative/result.csv",index =None)

In [28]:
from joblib import dump, load
dump(clf1,"svm_model/MID.joblib")

['svm_model/MID.joblib']

In [29]:
clf_curr = load('svm_model/MID.joblib')

In [48]:
infer_path = "../Data/chat_pattern/gpt_iterative/inference/pattern_discovery/995/chat_annotation_995.csv"
pattern_path = "../Data/chat_pattern/gpt_iterative/inference/pattern_discovery/995/etd_pattern_infer.csv"
df_result = evaluation_report(clf1, infer_path, pattern_path)

In [49]:
df_result

,Id,source,issue,POS,ETD_sent,Pattern for ETD,predicted_ETD,Comment_ETD,y_ETD,y'_ETD,...,PS_sent,Pattern for PS,predicted_PS,Comment_PS,y_PS,y'_PS,accuracy_PS,TS_sent,y_prob,y_pred
0,7,angular,"Hey guys, I'm trying to set a property in my c...","Hey-UH guys-NNS , I-PRP 'm-VBP trying-VBG to-T...",T1: I'm trying to set a property in my compone...,ETD_TRYING_TO,"ETD_TRYING_TO,",NaN,1,1,...,But everywhere I try to do it messes with chan...,PS_PROBLEM,"PS_PROBLEM,",NaN,1,1,1,NaN,0,0
1,14,angular,"Hi, I'm following tutorials from angular.io an...","Hi-UH , I-PRP 'm-VBP following-VBG tutorials-N...",Can I configure building anyhow that I don't h...,ETD_CAN_QUESTION,"ETD_CAN_QUESTION,",NaN,1,1,...,T1: When I move it somewhere else I have probl...,PS_PROBLEM,"PS_NEG_ADV_ADJ,PS_NEG_AUX_VERB,PS_VERB_ERROR,P...",NaN,1,1,1,NaN,0,0
2,21,angular,Trying to come up with a generic pattern valid...,Trying-VBG to-TO come-VB up-RP with-IN a-DT ge...,T1: Trying to come up with a generic pattern v...,ETD_TRYING_TO,"ETD_INSTEAD_OF_EXP_BEHAVIOR,ETD_TRYING_TO,",NaN,1,1,...,T2: but running into an issue of only getting ...,PS_PROBLEM,"PS_PROBLEM,PS_NO_NOUN,PS_ONLY,",NaN,1,1,1,NaN,0,0
3,34,angular,I'm sure this has been asked plenty before but...,I-PRP 'm-VBP sure-JJ this-DT has-VBZ been-VBN ...,T1: now looking to move to Angular2,ETD_LOOKING_TO,"ETD_LOOKING_TO,",NaN,1,1,...,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0
4,105,angular,Does anyone know why there is anngSubmitEventE...,Does-VBZ anyone-NN know-VB why-WRB there-EX is...,NaN,NaN,"ETD_LOOKING_TO,",NaN,0,1,...,T1: But I don't see any reason why there is a ...,"PS_NEG_AUX_VERB,PS_ONLY","PS_NEG_AUX_VERB,",NaN,1,1,1,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,22204,ethereum,"Hello, a transaction has given me gas error.Bu...","Hello-UH , a-DT transaction-NN has-VBZ given-V...",NaN,NaN,"ETD_CAN_QUESTION,",NaN,0,1,...,a transaction has given me gas error,"PS_NEG_AUX_VERB,PS_NEG_ADV_ADJ,","PS_NEG_AUX_VERB,PS_NEG_ADV_ADJ,",NaN,1,1,1,NaN,0,0
991,29720,typescript,Hello [<-CODE->] Here I have [<-CODE->] Erro...,Hello-UH [--LRB- <-SYM --HYPH CODE-NN --HYPH >...,NaN,NaN,NaN,NaN,0,0,...,"Here I have [<-CODE->] Error:(10, 16)","PS_NEG_AUX_ADV_ADJ,PS_PROBLEM,PS_NO_NOUN,PS_VE...","PS_NEG_AUX_ADV_ADJ,PS_PROBLEM,PS_NO_NOUN,PS_VE...",NaN,1,1,1,NaN,0,0
992,6042,appium,org.openqa.selenium.WebDriverException: org.ap...,org.openqa.selenium.WebDriverException-NN : or...,NaN,NaN,NaN,NaN,0,0,...,failed: Connection refused: connect Getting th...,"PS_PROBLEM,PS_NEG_AUX_VERB,PS_NEG_VERB,PS_ERRO...","PS_PROBLEM,PS_NEG_AUX_VERB,PS_NEG_VERB,PS_ERRO...",NaN,1,1,1,NaN,0,0
993,18124,docker,Not sure what this error is about: Error resp...,Not-RB sure-JJ what-WP this-DT error-NN is-VBZ...,NaN,NaN,"ETD_CAN_QUESTION,ETD_FOR_EXAMPLE,",NaN,0,1,...,Error response from daemon: rpc error,"PS_VERB_ERROR,PS_ERROR_NOUN_PHRASE,","PS_VERB_ERROR,PS_ERROR_NOUN_PHRASE,",NaN,1,1,1,NaN,0,0


In [9]:
# 2. ETD: pos
clf2,etd2 = cross_validate(X_pos_train_list, y_etd_train_list, X_pos_test_list, y_etd_test_list, X_pos_val_list, y_etd_val_list)

Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 10.527210526315788}
precision: 0.572 recall: 0.241 F1: 0.339


In [10]:
# 3. ETD: n-gram
clf3,etd3 = cross_validate(X_ngram_train_list, y_etd_train_list, X_ngram_test_list, y_etd_test_list,X_ngram_val_list, y_etd_val_list)


Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.588 recall: 0.305 F1: 0.402


In [11]:
# 4. ETD: pattern + pos
clf4,etd4 = cross_validate(X_etd_pos_train_list, y_etd_train_list, X_etd_pos_test_list, y_etd_test_list,X_etd_pos_val_list, y_etd_val_list)


Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.617 recall: 0.338 F1: 0.437


In [12]:
# 5. ETD: pattern + n-gram
clf5,etd5 = cross_validate(X_etd_ngram_train_list, y_etd_train_list, X_etd_ngram_test_list, y_etd_test_list, X_etd_ngram_val_list, y_etd_val_list)


Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
precision: 0.602 recall: 0.591 F1: 0.597


In [13]:
# 6. ETD: n-gram + pos
clf6,etd6 = cross_validate(X_ngram_pos_train_list, y_etd_train_list, X_ngram_pos_test_list, y_etd_test_list,X_ngram_pos_val_list, y_etd_val_list)


Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.643 recall: 0.451 F1: 0.53


In [14]:
# 7. ETD: pattern + n-gram + pos
clf7,etd7 = cross_validate(X_all_etd_train_list, y_etd_train_list, X_all_etd_test_list, y_etd_test_list,X_all_etd_val_list, y_etd_val_list)


Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.644 recall: 0.463 F1: 0.539


In [15]:
# 8. PS: pattern
clf8,ps8 =cross_validate(X_ps_train_list, y_ps_train_list, X_ps_test_list, y_ps_test_list, X_ps_val_list, y_ps_val_list)

Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 10.527210526315788}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
precision: 0.735 recall: 0.748 F1: 0.741


In [16]:
# 9. PS: pos
clf9,ps9 = cross_validate(X_pos_train_list, y_ps_train_list, X_pos_test_list, y_ps_test_list,X_pos_val_list, y_ps_val_list)

Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.587 recall: 0.4 F1: 0.476


In [17]:
# 10. PS: n-gram
clf10,ps10 = cross_validate(X_ngram_train_list, y_ps_train_list, X_ngram_test_list, y_ps_test_list,X_ngram_val_list, y_ps_val_list)

Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.474 recall: 0.822 F1: 0.602


In [18]:
# 11. PS: pattern + pos
clf11,ps11 = cross_validate(X_ps_pos_train_list, y_ps_train_list, X_ps_pos_test_list, y_ps_test_list,X_ps_pos_val_list, y_ps_val_list)


Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.698 recall: 0.522 F1: 0.597


In [19]:
# 12. PS: pattern + n-gram
clf12,ps12 = cross_validate(X_ps_ngram_train_list, y_ps_train_list, X_ps_ngram_test_list, y_ps_test_list,X_ps_ngram_val_list, y_ps_val_list)


Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
precision: 0.571 recall: 0.922 F1: 0.705


In [20]:
# 13. PS: n-gram + pos
clf13,ps13 = cross_validate(X_ngram_pos_train_list, y_ps_train_list, X_ngram_pos_test_list, y_ps_test_list,X_ngram_pos_val_list, y_ps_val_list)


Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 0.001}
precision: 0.608 recall: 0.489 F1: 0.542


In [21]:
# 14. PS: pattern + n-gram + pos
clf14,ps14 = cross_validate(X_all_ps_train_list, y_ps_train_list, X_all_ps_test_list, y_ps_test_list,X_all_ps_val_list, y_ps_val_list)


Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 5.264105263157894}
Best parameters: {'C': 0.001}
Best parameters: {'C': 5.264105263157894}
precision: 0.681 recall: 0.656 F1: 0.668
